Pull the RABIES image from Docker hub https://hub.docker.com/r/gabdesgreg/rabies. Here we select the version 0.4.8

In [1]:
!docker pull gabdesgreg/rabies:0.4.8

0.4.8: Pulling from gabdesgreg/rabies

7da0a88c: Already exists 
a5f15c7a: Already exists 
be87baa1: Already exists 
26eae209: Pulling fs layer 
48f25545: Pulling fs layer 
14b5f91a: Pulling fs layer 
9c6fce70: Pulling fs layer 
a348ca3c: Pulling fs layer 
b25c3815: Pulling fs layer 
7b94ca36: Pulling fs layer 
30beeafe: Pulling fs layer 
4d1f9573: Pulling fs layer 
b1324eac: Pulling fs layer 
2f65a254: Pulling fs layer 
492d75f6: Pulling fs layer 
f4c6fb92: Pulling fs layer 
c6fce70: Waiting fs layer 
25c3815: Waiting fs layer 
b94ca36: Waiting fs layer 
b1324eac: Waiting fs layer 
2f65a254: Waiting fs layer 
f4c6fb92: Waiting fs layer 
8f16a8b3: Waiting fs layer 
5683cf99: Waiting fs layer 
ce1ecc28: Pulling fs layer 
Digest: sha256:f5517c1d7accbd3a398a0fb3fabd210727667300f32fb0dc909a6aaae89667ceDownloading  186.6MB/683MBDownloading  275.8MB/277.4MBDownloading    874MB/1.34GB
Status: Downloaded newer image for gabdesgreg/rabies:0.4.8
docker.io/gabdesgreg/rabies:0.4.8


Download test dataset (can be downloaded from here https://zenodo.org/record/8349029)

In [1]:
!curl -L --retry 5 https://zenodo.org/record/8349029/files/test_dataset.zip -o test_dataset.zip
!unzip test_dataset.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 42.5M  100 42.5M    0     0   532k      0  0:01:21  0:01:21 --:--:--  592k
Archive:  test_dataset.zip
   creating: test_dataset/
   creating: test_dataset/sub-PHG001/
   creating: test_dataset/sub-PHG001/ses-3/
   creating: test_dataset/sub-PHG001/ses-3/anat/
  inflating: test_dataset/sub-PHG001/ses-3/anat/sub-PHG001_ses-3_acq-RARE_T2w.json  
  inflating: test_dataset/sub-PHG001/ses-3/anat/sub-PHG001_ses-3_acq-RARE_T2w.nii.gz  
   creating: test_dataset/sub-PHG001/ses-3/func/
  inflating: test_dataset/sub-PHG001/ses-3/func/sub-PHG001_ses-3_task-rest_acq-EPI_run-1_bold.json  
  inflating: test_dataset/sub-PHG001/ses-3/func/sub-PHG001_ses-3_task-rest_acq-EPI_run-1_bold.nii.gz  
   creating: test_dataset/sub-PHG002/
   creating: test_dataset/sub-PHG002/ses-3/
   creating: test_dataset/sub-PHG002/ses-3/anat/
  inflating: test_data

In [2]:
!rm test_dataset.zip

# Preprocessing
* `--anatomical_resampling 0.15x0.15x0.15`: anatomical images and template files will be resampled to 0.15mm isotropic resolution to speed up registration (instead of using the original 0.1 isotropic resolution)
* `--anat_inho_cor method=SyN,otsu_thresh=2,multiotsu=true`: for inhomogeneity correction of anatomical images, we will be activating the multiotsu option (the other options are set to the default), which improves correction for pronounced inhomogeneity gradients (which was the case for this dataset)

In [4]:
# create the output folder,and allow for writing permissions so that Docker can write new outputs inside
!mkdir -p test_dataset_outputs
!chmod 777 test_dataset_outputs
!rm -f test_dataset_outputs/rabies_preprocess.pkl

!docker run -it --rm \
-v $PWD/test_dataset:/test_dataset:ro \
-v $PWD/test_dataset_outputs:/test_dataset_outputs/ \
gabdesgreg/rabies:0.4.8 -p MultiProc --local_threads 4 \
preprocess /test_dataset/ /test_dataset_outputs --anatomical_resampling 0.15x0.15x0.15 \
--anat_inho_cor method=SyN,otsu_thresh=2,multiotsu=true

230914-19:30:57,615 nipype.workflow INFO:
	 Running RABIES - version: 0.4.8
230914-19:30:57,615 nipype.workflow INFO:
	 CLI INPUTS: 
-> rabies_stage = preprocess 
-> plugin = MultiProc 
-> local_threads = 4 
-> scale_min_memory = 1.0 
-> min_proc = 1 
-> verbose = 1 
-> bids_dir = /test_dataset 
-> output_dir = /test_dataset_outputs 
-> bold_only = False 
-> anat_autobox = False 
-> bold_autobox = False 
-> apply_despiking = False 
-> HMC_option = intraSubjectBOLD 
-> apply_slice_mc = False 
-> detect_dummy = False 
-> data_type = float32 
-> anat_inho_cor = {'method': 'SyN', 'otsu_thresh': '2', 'multiotsu': True} 
-> anat_robust_inho_cor = {'apply': False, 'masking': False, 'brain_extraction': False, 'template_registration': 'SyN'} 
-> bold_inho_cor = {'method': 'Rigid', 'otsu_thresh': '2', 'multiotsu': False} 
-> bold_robust_inho_cor = {'apply': False, 'masking': False, 'brain_extraction': False, 'template_registration': 'SyN'} 
-> commonspace_reg = {'masking': False, 'brain_extracti

In [11]:
!rm -f test_dataset_outputs/rabies_confound_correction.pkl
!docker run -it --rm \
-v $PWD/test_dataset:/test_dataset:ro \
-v $PWD/test_dataset_outputs:/test_dataset_outputs/ \
gabdesgreg/rabies:0.4.8 -p MultiProc --local_threads 4 \
confound_correction /test_dataset_outputs/ /test_dataset_outputs \
--frame_censoring FD_censoring=true,FD_threshold=0.05,DVARS_censoring=false,minimum_timepoint=80 \
--conf_list mot_6 WM_signal CSF_signal --smoothing_filter 0.3

230914-21:03:04,797 nipype.workflow INFO:
	 Running RABIES - version: 0.4.8
230914-21:03:04,797 nipype.workflow INFO:
	 CLI INPUTS: 
-> rabies_stage = confound_correction 
-> plugin = MultiProc 
-> local_threads = 4 
-> scale_min_memory = 1.0 
-> min_proc = 1 
-> verbose = 1 
-> preprocess_out = /test_dataset_outputs 
-> output_dir = /test_dataset_outputs 
-> nativespace_analysis = False 
-> image_scaling = global_variance 
-> detrending_order = linear 
-> conf_list = ['mot_6', 'WM_signal', 'CSF_signal'] 
-> frame_censoring = {'FD_censoring': True, 'FD_threshold': 0.05, 'DVARS_censoring': False, 'minimum_timepoint': 80} 
-> TR = auto 
-> highpass = None 
-> lowpass = None 
-> edge_cutoff = 0 
-> smoothing_filter = 0.3 
-> match_number_timepoints = False 
-> ica_aroma = {'apply': False, 'dim': 0, 'random_seed': 1} 
-> read_datasink = False 
-> timeseries_interval = all 

230914-21:03:06,15 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.6.0
2

In [12]:
!rm -f test_dataset_outputs/rabies_analysis.pkl
!docker run -it --rm \
-v $PWD/test_dataset:/test_dataset:ro \
-v $PWD/test_dataset_outputs:/test_dataset_outputs/ \
gabdesgreg/rabies:0.4.8 -p MultiProc --local_threads 4 \
analysis /test_dataset_outputs/ /test_dataset_outputs \
--data_diagnosis

230914-21:03:59,609 nipype.workflow INFO:
	 Running RABIES - version: 0.4.8
230914-21:03:59,609 nipype.workflow INFO:
	 CLI INPUTS: 
-> rabies_stage = analysis 
-> plugin = MultiProc 
-> local_threads = 4 
-> scale_min_memory = 1.0 
-> min_proc = 1 
-> verbose = 1 
-> confound_correction_out = /test_dataset_outputs 
-> output_dir = /test_dataset_outputs 
-> scan_list = ['all'] 
-> prior_maps = /home/rabies/.local/share/rabies/melodic_IC.nii.gz 
-> prior_bold_idx = [5, 12, 19] 
-> prior_confound_idx = [0, 1, 2, 6, 7, 8, 9, 10, 11, 13, 14, 21, 22, 24, 26, 28, 29] 
-> data_diagnosis = True 
-> outlier_threshold = 3.5 
-> seed_list = [] 
-> seed_prior_list = [] 
-> FC_matrix = False 
-> ROI_type = parcellated 
-> ROI_csv = /home/rabies/.local/share/rabies/DSURQE_40micron_labels.nii.gz 
-> group_ica = {'apply': False, 'dim': 0, 'random_seed': 1} 
-> DR_ICA = False 
-> NPR_temporal_comp = -1 
-> NPR_spatial_comp = -1 
-> network_weighting = absolute 

/home/rabies/miniforge/lib/python3.9/sit